## Author: Min Shi
## Last updated: 5/18/2021
## Description:
The code was created to implement the NetOIF model to impute RPPA data.

In [2]:
import time
import tensorflow as tf
import numpy as np
from models import gcn, lstm
from configs import *
from utils import *
import scipy.sparse
import statistics

It performs the feature imputation task in which all historical features 
before time t are used to impute features at time t in a supervised manner.
The adopted method concatenates GCN and LSTM in a unified framework, where
GCN is utilized to model the PPI network and LSTM is utilized to mdoel the 
time-serial features. 

# 1. Load the RPPA data

* **`RPPA_level1.xlsx:`** Reverse-Phase Protein Array, representing the ligand-protein regulations at 1,4,8,24 and 48 hour
* **`string_interactions.tsv:`** The protein-protein interaction (PPI) network from stringDB

In [ ]:
np.random.seed(123)
FLAGS = tf.flags.FLAGS
dataset = FLAGS.rppa
time_steps = 5
hidden_dim = 6
hidden_size = 6
train_ratio = FLAGS.train_ratio
num_run = FLAGS.num_run
train_ratio

In [4]:
feat_file = os.path.join('datasets/rppa', 'MDD_RPPA_Level3_preprocessed_2020-9.xlsx')
feat_df = pd.read_excel(feat_file, sheet_name='MDD_RPPA_Level3_annotated').set_index('Protein')
feat_df.columns = [c.split('_')[0]+'_'+c.split('_')[1] for c in feat_df.columns]
feat_df = feat_df.loc[:,~feat_df.columns.str.startswith('Ctrl')]
feat_df = feat_df.apply(pd.to_numeric, errors='ignore')
feat_df = feat_df.groupby(feat_df.columns, axis=1, sort=False).mean()

feats = feat_df.values
feats = feats.reshape([feats.shape[0],6,5]).transpose([2, 0, 1])
feat_list = []
for i in range(time_steps):
    feat_list.append(feats[i])
#         feat_list.append(sp.coo_matrix(feats[i]))

#     print(feats.shape)

pnames = feat_df.index.tolist()

network_file = os.path.join('datasets/rppa', 'string_interactions_new.tsv')
net_df = pd.read_csv(network_file,sep='\t')

adj = np.zeros([len(pnames),len(pnames)])
links = []
no_links = []
for index, row in net_df.iterrows():
    try:
        v1_idx = pnames.index(row['node1'])
        v2_idx = pnames.index(row['node2'])
        score = float(row['combined_score'])

        adj[v1_idx,v2_idx] = score
        adj[v2_idx,v1_idx] = score

        links.append((row['node1'], row['node2']))
    except:
        no_links.append((row['node1'], row['node2']))

In [ ]:
print('Adjacency matrix:'+str(adj.shape))
print('Time series rppa data:'+str(feats.shape))

In [ ]:
# adjs, feats, train_mask, val_mask, test_mask = load_data_impute(dataset, time_steps, train_ratio)
# adjs, feats, train_masks, val_masks, test_masks, protein_names, ligand_names = load_data_impute(dataset, time_steps, 
#                                                                                                 train_ratio, num_run)
adjs, feats, train_masks, val_masks, test_masks, protein_names, ligand_names = load_data_impute(dataset, time_steps, 
                                                                                                train_ratio, num_run,
                                                                                               True, sparse_rate = 0.5)
train_mask, val_mask, test_mask = [train_masks[num_run-1], val_masks[num_run-1], test_masks[num_run-1]]

num_node = adjs[0].shape[0]
num_feat = feats[0].shape[1]
for i in range(time_steps):
    adjs[i] = sparse_to_tuple(scipy.sparse.coo_matrix(adjs[i]))
#     feats[i] = sparse_to_tuple(scipy.sparse.coo_matrix(feats[i]))
num_features_nonzeros = [x[1].shape for x in feats]

# 2. Prepare the model

In [10]:
alpha = 0.1

# define placeholders of the input data 
phs = {
        'adjs': [tf.sparse_placeholder(tf.float32, shape=(None, None), name="adjs") for i in
             range(time_steps)],
        'feats': [tf.placeholder(tf.float32, shape=(None, num_feat), name="feats") for _ in
                 range(time_steps)],
        'train_mask': tf.placeholder(tf.float32, shape=(None,None), name="train_mask"),
        'val_mask': tf.placeholder(tf.float32, shape=(None,None), name="val_mask"),
        'test_mask': tf.placeholder(tf.float32, shape=(None,None), name="test_mask"),
        'sample_idx': tf.placeholder(tf.int32, shape=(FLAGS.batch_size,), name='batch_sample_idx'),
        'dropout_prob': tf.placeholder_with_default(0., shape=()),
        'num_features_nonzeros': [tf.placeholder(tf.int64) for i in range(time_steps)]
        }

# define the GCN model
gcn_model = gcn.GraphConvLayer(time_steps = time_steps,
                               gcn_layers=FLAGS.gcn_layers,
                               input_dim=num_feat,
                               hidden_dim=hidden_dim,
                               output_dim=hidden_size,
                               name='nn_fc1',
                               num_features_nonzeros=phs['num_features_nonzeros'],
                               act=tf.nn.relu,
                               dropout_prob=phs['dropout_prob'],
                               dropout=True)

embeds_list = gcn_model(adjs=phs['adjs'],
                    feats=phs['feats'],
                    sparse=False)

# prepare train data for the LSTM-based prediction model
## replace all missing features at (time_steps-1) with GCN imputed features
# embeds_list[time_steps-1] = tf.add(phs['feats'][time_steps-1], 
#                                    tf.multiply(phs['test_mask'][time_steps-1], embeds_list[time_steps-1]))
## construct training samples for the prediction task
combined_feats = []
for i in range(time_steps):
    combined_feats.append(alpha*tf.add(phs['feats'][i], (1-alpha)*embeds_list[i]))

x_input, y_label = build_train_samples_imputation(embeds_list=combined_feats, 
                                                     feats=phs['feats'], 
                                                     time_steps=time_steps)
# define the bi-directional LSTM model
lstm_model = lstm.BiLSTM(hidden_size=hidden_size,
                         seq_len=FLAGS.time_steps-1,
                         holders=phs)
x_input_seq = tf.gather(x_input, phs['sample_idx'])
y_input_seq_real = tf.gather(y_label, phs['sample_idx'])
y_input_seq_mask = tf.gather(phs['train_mask'], phs['sample_idx'])
y_input_seq_pred = lstm_model(input_seq=x_input_seq)


with tf.name_scope('optimizer'):
    # calculate the train mse and ad
    train_mse = tf.losses.mean_squared_error(tf.multiply(y_input_seq_real,y_input_seq_mask), 
                                             tf.multiply(y_input_seq_pred,y_input_seq_mask))
    train_absolute_diff = tf.losses.absolute_difference(tf.multiply(y_input_seq_real,y_input_seq_mask), 
                                             tf.multiply(y_input_seq_pred,y_input_seq_mask))
    
    # calculate the val mse and ad
    val_input_seq_pred = lstm_model(input_seq=x_input)
    val_mse = tf.losses.mean_squared_error(tf.multiply(y_label, phs['val_mask']), 
                                           tf.multiply(val_input_seq_pred, phs['val_mask']))
    val_absolute_diff = tf.losses.absolute_difference(tf.multiply(y_label, phs['val_mask']), 
                                           tf.multiply(val_input_seq_pred, phs['val_mask']))
    
    # calculate the test mse and ad
    test_input_seq_pred = lstm_model(input_seq=x_input)
    test_mse = tf.losses.mean_squared_error(tf.multiply(y_label, phs['test_mask']), 
                                            tf.multiply(test_input_seq_pred, phs['test_mask']))
    test_absolute_diff = tf.losses.absolute_difference(tf.multiply(y_label, phs['test_mask']), 
                                            tf.multiply(test_input_seq_pred, phs['test_mask']))
    
    missing_actual = tf.multiply(y_label, phs['test_mask'])
    missing_predicted = tf.multiply(test_input_seq_pred, phs['test_mask'])
    
    optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)
    opt_op = optimizer.minimize(train_mse)

n_cpus = 8
config = tf.ConfigProto(device_count={ "CPU": n_cpus},
                            inter_op_parallelism_threads=n_cpus,
                            intra_op_parallelism_threads=2)

feed_dict = {phs['train_mask']: train_mask,
             phs['val_mask']: val_mask,
             phs['test_mask']: test_mask,
             phs['sample_idx']: None,
             phs['dropout_prob']: FLAGS.dropout_prob}

feed_dict.update({phs['adjs'][t]: adjs[t] for t in range(time_steps)})
feed_dict.update({phs['feats'][t]: feats[t] for t in range(time_steps)})
feed_dict.update({phs['num_features_nonzeros'][t]: num_features_nonzeros[t] for t in range(time_steps)})

feed_dict_val = {phs['train_mask']: train_mask,
                 phs['val_mask']: val_mask,
                 phs['test_mask']: test_mask,
                 phs['dropout_prob']: 0.}

feed_dict_val.update({phs['adjs'][t]: adjs[t] for t in range(time_steps)})
feed_dict_val.update({phs['feats'][t]: feats[t] for t in range(time_steps)})
feed_dict_val.update({phs['num_features_nonzeros'][t]: num_features_nonzeros[t] for t in range(time_steps)})

def get_batch_idx(epoch):
    s = FLAGS.batch_size * epoch
    e = FLAGS.batch_size * (epoch + 1)
    idx = []
    for i in range(s,e):
        idx.append(i%num_node)
    return idx

# 3. run the model

In [ ]:
epochs = FLAGS.epochs
save_step = 10
t = time.time()

test_MSEs = []
test_ADs = []
for k in range(num_run):
    train_mask_, val_mask_, test_mask_ = [train_masks[k], val_masks[k], test_masks[k]]
    feed_dict.update({phs['train_mask']:train_mask_,
                     phs['val_mask']: val_mask_,
                     phs['test_mask']: test_mask_})
    feed_dict_val.update({phs['train_mask']:train_mask_,
                     phs['val_mask']: val_mask_,
                     phs['test_mask']: test_mask_})

    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        batch_samples = get_batch_idx(epoch)
        feed_dict.update({phs['sample_idx']: batch_samples})
        _, train_MSE, train_AD = sess.run((opt_op, train_mse, train_absolute_diff), feed_dict=feed_dict)
        val_MSE, val_AD = sess.run((val_mse, val_absolute_diff), 
                                             feed_dict=feed_dict_val) 
        
#         print("Epoch:", '%04d' % (epoch + 1),
#           "train_loss=", "{:.5f}".format(train_MSE),
#           "train_MSE=", "{:.5f}".format(train_MSE),
#           "train_AD=", "{:.5f}".format(train_AD),
#           "val_MSE=", "{:.5f}".format(val_MSE),
#           "val_AD=", "{:.5f}".format(val_AD),
#           "time=", "{:.5f}".format(time.time() - t))
        
#         if (epoch+1) % save_step == 0:
    test_MSE, test_AD, embeds_list_out, missing_actual_, missing_predicted_ = sess.run((test_mse, test_absolute_diff, 
                                                                   embeds_list, missing_actual, missing_predicted), 
                                                                   feed_dict=feed_dict_val) 
    
    print("run={}-------test_MSE=".format(k+1), 
          "{:.5f}".format(test_MSE),
          "test_AD=", "{:.5f}".format(test_AD))
    
    test_MSEs.append(float(test_MSE))
    test_ADs.append(float(test_AD))
            
average_MSE = statistics.mean(test_MSEs)
stdev_MSE = statistics.stdev(test_MSEs)
average_AD = statistics.mean(test_ADs)
stdev_AD = statistics.stdev(test_ADs)
print('average_MSE=%f, stdev_MSE=%f, average_AD=%f, stdev_AD=%f' % (average_MSE, stdev_MSE,
                                                                      average_AD, stdev_AD))   

In [ ]:
import matplotlib.ticker as plticker
from matplotlib import colors
import matplotlib.pyplot as plt

protein_names = np.array(protein_names)

fig = plt.figure(figsize=(4,6))
fig.subplots_adjust(hspace=0, wspace=0.4)
ax1 = fig.add_subplot(121)
ax1.set_aspect('auto')
cax1 = ax1.matshow(missing_actual_,vmin=-2,vmax=2,cmap='bwr',norm = colors.DivergingNorm(vcenter=0), aspect="auto")
fig.colorbar(cax1)
ax1.set_title('Actual_48h',y=-0.1, fontsize=11)
ax1.set_xticklabels(range(7),fontsize=11)

ax1.set_yticks(range(len(protein_names)))
loc = plticker.MultipleLocator(base=10) # this locator puts ticks at regular intervals
ax1.yaxis.set_major_locator(loc)
show_protein_names = protein_names[range(0, len(protein_names), 10)]
show_protein_names = np.insert(show_protein_names, 0, 0, axis=0)
# ligand_names = np.insert(ligand_names, 0, 0, axis=0)
ax1.set_yticklabels(show_protein_names, fontsize=11)
ax1.set_xticklabels(ligand_names, fontsize=11, rotation=90)


fig1 = plt.figure(figsize=(4,6))
fig1.subplots_adjust(hspace=0, wspace=0.4)

ax2 = fig1.add_subplot(122)
ax2.set_aspect('auto')
cax2 = ax2.matshow(missing_predicted_,vmin=-2,vmax=2,cmap='bwr',norm = colors.DivergingNorm(vcenter=0), aspect="auto")
fig1.colorbar(cax2)
ax2.set_title('Imputed_48h',y=-0.1, fontsize=11)
ax2.set_xticklabels(range(7),fontsize=11)

ax2.set_yticks(range(len(protein_names)))
ax2.yaxis.set_major_locator(loc)
ax2.set_yticklabels(show_protein_names, fontsize=11)
ax2.set_xticklabels(ligand_names, fontsize=11, rotation=90)